# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
MODEL_GPT = 'gpt-4o-mini'
#MODEL_LLAMA = 'llama3.2'

API key looks good so far


In [3]:
openai = OpenAI()

In [12]:
# set up environment
system_prompt= " You are a tutor / coding assistant which deals with the technical / coding questions and provides answers / explanations "
my_question = input("Please enter your question:")


Please enter your question: Which is the most powerful LLM ?


In [13]:
# here is the question; type over this to ask something ne
def get_user_prompt(question):
    user_prompt = "The question for you is as below. Please provide an accurate response / explanation"
    user_prompt += "".join(question)
    return user_prompt

In [14]:
def get_message(question):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(question)}
          ]
    return messages

In [15]:
# Get gpt-4o-mini to answer, with streaming

def get_answers_from_openai(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=get_message(question),
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [16]:
get_answers_from_openai(my_question)
#"yield from {book.get(\"author\") for book in books if book.get(\"author\")}"

As of my last update in October 2023, the term "most powerful LLM" (Large Language Model) can be subjective and context-dependent, as various models excel in different areas, such as size, training data, architecture, and specific capabilities.

1. **OpenAI's GPT-4**: This model is among the most powerful and widely known LLMs. It features improved understanding, reasoning, and generation capabilities compared to its predecessors, including GPT-3.5. It's designed to handle a wide range of tasks, from text generation to code completion and natural language understanding.

2. **Google's PaLM (Pathways Language Model)**: Another contender released by Google, PaLM has also been noted for its size and capabilities, particularly in tasks involving natural language understanding and generation. The PaLM series is developed to be highly flexible and capable of multitasking.

3. **Meta's LLaMA (Large Language Model Meta AI)**: This model focuses on being efficient and effective, and it has shown competitive performance in various natural language processing (NLP) benchmarks. LLaMA serves as an open-source alternative to proprietary LLMs.

4. **Anthropic's Claude**: This is also a highly regarded model aimed at safety and ethical considerations in language understanding and generation.

5. **Cohere's and other emerging models**: There are various other LLMs developed by different organizations that target specific use cases or aim at optimizing the deployment of models in real-time scenarios.

Ultimately, "most powerful" depends on factors such as the model's architecture, training method, specific tasks it's evaluated on, the dataset used for training, and user requirements for speed, efficiency, and ethical considerations. Advances are continually being made, so the landscape can change rapidly. For the latest comparisons on performance and capability, examining benchmark results and scientific publications can provide deeper insights.

In [ ]:
MODEL_LLAMA = 'llama3.2'

In [ ]:
# Get Llama 3.2 to answer
def get_answers_from_ollama(question):
    response = ollama.chat(model=MODEL_LLAMA, messages=get_message(question))
    reply = response['message']['content']
    print(reply)
    display(Markdown(reply))

In [ ]:
#response = ollama.chat(model=MODEL_LLAMA, messages=get_message("What is the best way to learn AI ? Can you provide a roadmap ?"))
#reply = response['message']['content']
#display(Markdown(reply))

In [ ]:
#get_answers_from_ollama("What is the best way to learn AI ? Can you provide a roadmap ?")